In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

Common loss functions include:

nn.MSELoss (Mean Square Error) used for regression tasks

nn.NLLLoss (Negative Log Likelihood) used for classification

nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303963  [    0/60000]
loss: 2.296804  [ 6400/60000]
loss: 2.286106  [12800/60000]
loss: 2.278895  [19200/60000]
loss: 2.274703  [25600/60000]
loss: 2.249207  [32000/60000]
loss: 2.252695  [38400/60000]
loss: 2.232362  [44800/60000]
loss: 2.232735  [51200/60000]
loss: 2.198218  [57600/60000]
Test Error: 
 Accuracy: 44.7%, Avg loss: 0.034707 

Epoch 2
-------------------------------
loss: 2.228429  [    0/60000]
loss: 2.220963  [ 6400/60000]
loss: 2.171648  [12800/60000]
loss: 2.165574  [19200/60000]
loss: 2.171803  [25600/60000]
loss: 2.109218  [32000/60000]
loss: 2.133925  [38400/60000]
loss: 2.086202  [44800/60000]
loss: 2.108560  [51200/60000]
loss: 2.028609  [57600/60000]
Test Error: 
 Accuracy: 51.6%, Avg loss: 0.032310 

Epoch 3
-------------------------------
loss: 2.108470  [    0/60000]
loss: 2.087003  [ 6400/60000]
loss: 1.980889  [12800/60000]
loss: 1.974307  [19200/60000]
loss: 2.005027  [25600/60000]
loss: 1.886485  [32000/600

In [8]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
